In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mplt
import seaborn as sns
import arviz as az
import datetime

In [ ]:
from cmdstanpy import cmdstan_path, CmdStanModel


In [ ]:
def open():
    df = pd.read_csv('data.csv')
    df.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
    df.set_index('Date', inplace=True)
    df.fillna(np.NaN, inplace=True)
    
    fields = ["alt", "tipo", "zoning"]
    coordinates = ["latitude","longitude"]
    coord_matrix= pd.read_csv('coord.csv',sep=",",names=coordinates)
    
    reg_matrix=pd.read_csv('regression.csv',sep=";",names=fields)
    
    dummies = pd.get_dummies(reg_matrix, columns=['tipo','zoning'], drop_first=True)
    
    
    return (df,coord_matrix,dummies)

In [ ]:
ritorno= open()
reg_matrix=ritorno[2]
coord_matrix=ritorno[1]
coord_matrix.dtypes

latitude     float64
longitude    float64
dtype: object

In [ ]:
def compute(p, q, samples_per_chain = 1250, catene = 4, burnin = 1000):

    ritorno = open()
    df = ritorno[0]
    coord_matrix=ritorno[1]
    reg_matrix = ritorno[2]
    data = {
        "T":df.shape[0],
        "S":df.shape[1],
        "reg":reg_matrix.shape[1],
        "p":int(p),
        "q":int(q),
        "y":np.nan_to_num(df.to_numpy(), nan=1.0),
        "X":reg_matrix,
        "is_missing":np.isnan(df.to_numpy()).astype(int),
        "missing_size": np.sum(np.isnan(df.to_numpy()).astype(int)),
        "coord":coord_matrix
    }        
    #stan_model = CmdStanModel(exe_file='./code.exe')
    stan_model= CmdStanModel(stan_file='spatial.stan')
    stan_fit = stan_model.sample(data=data, chains=catene, 
                                 parallel_chains=catene, 
                                 iter_warmup=burnin,  iter_sampling=samples_per_chain)
    inference_data = az.from_cmdstanpy(stan_fit)

    """
    missing_entries = np.isnan(df[stazione].to_numpy())
    y_missing = inference_data.posterior.y_missing.values[:,:,missing_entries]
    date_mancanti = df.index[missing_entries]
    
    y_missing_dict = {}
    for i in range(len(date_mancanti)):
        y_missing_dict[str(date_mancanti[i])] = y_missing[:,:,i]
    
    return {'inference_data': inference_data, 'reconstructed_y': y_missing_dict}
    """
    return {'inference_data': inference_data}


## Day Planner
Il modello attualmente considerato è il seguente:

Definita la funzione $\text{sigmoide}$:
$$
\Sigma(z) = \frac{e^z - 1}{e^z + 1}
$$

Sia $s$ l'indice che scorre le stazioni:
$$
s \in \left\{ 1,\dots, S \right\}
$$
allora:
$$
\left. \mathbf{y}_s \right| \gamma_\theta, \underline{\gamma_\phi}, \sigma, \underline{\mu_\phi}, \underline{\sigma_\phi} \sim \text{ARIMA}_{2,1,1}\left( \Sigma(\gamma_{\phi,s}[0]), \Sigma(\gamma_{\phi,s}[1]), \Sigma(\gamma_\theta) , \sigma\right)\\
\left. \gamma_{\phi,s}[j] \right| \underline{\mu_\phi}, \underline{\sigma_\phi} \sim \mathcal{N}\left( \mu_{\phi}[j], \sigma_{\phi}[j] \right), \qquad j \in \left\{ 1,\dots,p \right\} = \left\{ 1,2 \right\}\\
\left. \gamma_\theta \right. \sim \mathcal{N}\left( 0,1 \right)\\
\left. \mu_\phi[j] \right. \sim \mathcal{N}\left( 0,5 \right) \qquad j \in \left\{ 1,\dots,p \right\}\\
\left. \sigma_\phi[j] \right. \sim \mathcal{IG}\left( 2.1,1.1 \right) \qquad j \in \left\{ 1,\dots,p \right\}
$$

L'ARIMA utilizza anche dati precedenti al giorno iniziale trattati come dati mancanti.
I dati mancanti in generale hanno bisogno di prior, che sono state scelte come segue:
$$
y[missing] \sim \mathcal{N}\left( 1,1 \right)
$$
(nel codice STAN sono la variabile 'w')

Tuttavia i dati precedenti i giorni iniziali sono stati trattati diversamente, secondo il modello:
$$
y_{start} | \mu_{start}, \sigma_{start} \sim \mathcal{N}\left( \mu_{start}, \sigma_{start} \right)\\
\mu_{start} \sim \mathcal{N}\left( 1,1 \right)\\
\sigma_{start} \sim \mathcal{IG}\left( 3,2 \right)
$$

Naturalmente se scrivessimo le cose per bene, la likelihood dell'ARIMA dovrebbe essere condizionata anche ai dati mancanti

In [ ]:
df_temp = open()
df = df_temp[0]

In [ ]:
import cmdstanpy
cmdstanpy.install_cmdstan()
cmdstanpy.install_cmdstan(compiler=True)  # only valid on Windows

Installing CmdStan version: 2.31.0
Install directory: /root/.cmdstan
Download successful, file: /tmp/tmpznhhetqs
Extracting distribution


DEBUG:cmdstanpy:cmd: make build -j1
cwd: None


Unpacked download as cmdstan-2.31.0
Building version cmdstan-2.31.0, may take several minutes, depending on your system.


DEBUG:cmdstanpy:cmd: make examples/bernoulli/bernoulli
cwd: None


Test model compilation
Installed cmdstan-2.31.0
Installing CmdStan version: 2.31.0
Install directory: /root/.cmdstan
CmdStan version 2.31.0 already installed


True

In [ ]:
from cmdstanpy import cmdstan_path, CmdStanModel
stan_model = CmdStanModel(stan_file='spatial.stan')

11:39:04 - cmdstanpy - INFO - compiling stan file /content/spatial.stan to exe file /content/spatial
INFO:cmdstanpy:compiling stan file /content/spatial.stan to exe file /content/spatial
DEBUG:cmdstanpy:cmd: make /content/spatial
cwd: /root/.cmdstan/cmdstan-2.31.0
DEBUG:cmdstanpy:Console output:

--- Translating Stan model to C++ code ---
bin/stanc  --o=/content/spatial.hpp /content/spatial.stan
Warning in '/content/spatial.stan', line 18, column 2: Declaration of arrays
    by placing brackets after a variable name is deprecated and will be
    removed in Stan 2.32.0. Instead use the array keyword before the type.
    This can be changed automatically using the auto-format flag to stanc
Warning in '/content/spatial.stan', line 213, column 42: Comments beginning
    with # are deprecated and this syntax will be removed in Stan 2.32.0. Use
    // to begin line comments; this can be done automatically using the
    auto-format flag to stanc
Warning in '/content/spatial.stan', line 109, c

In [ ]:
ritorno= compute(2, 1, catene=6, samples_per_chain=750, burnin=500)

DEBUG:cmdstanpy:found newer exe file, not recompiling
DEBUG:cmdstanpy:input tempfile: /tmp/tmp3xmh9l5n/of_fd2v2.json
DEBUG:cmdstanpy:cmd: /content/spatial info
cwd: None
11:40:18 - cmdstanpy - INFO - CmdStan start processing
INFO:cmdstanpy:CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

chain 5 |          | 00:00 Status

chain 6 |          | 00:00 Status

DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: 1
DEBUG:cmdstanpy:idx 1
DEBUG:cmdstanpy:idx 2
DEBUG:cmdstanpy:CmdStan args: ['/content/spatial', 'id=1', 'random', 'seed=50910', 'data', 'file=/tmp/tmp3xmh9l5n/of_fd2v2.json', 'output', 'file=/tmp/tmp3xmh9l5n/spatial3ne4131_/spatial-20230204114018_1.csv', 'method=sample', 'num_samples=750', 'num_warmup=500', 'algorithm=hmc', 'adapt', 'engaged=1']
DEBUG:cmdstanpy:idx 3
DEBUG:cmdstanpy:running CmdStan, num_threads: 1
DEBUG:cmdstanpy:idx 4
DEBUG:cmdstanpy:idx 5
DEBUG:cmdstanpy:running CmdStan, num_threads: 1
DEBUG:cmdstanpy:running CmdStan, num_threads: 1
DEBUG:cmdstanpy:CmdStan args: ['/content/spatial', 'id=4', 'random', 'seed=50910', 'data', 'file=/tmp/tmp3xmh9l5n/of_fd2v2.json', 'output', 'file=/tmp/tmp3xmh9l5n/spatial3ne4131_/spatial-20230204114018_4.csv', 'method=sample', 'num_samples=750', 'num_warmup=500', 'algorithm=hmc', 'adapt', 'engaged=1']
DEBUG:cmdstanpy:running CmdStan, num_threads: 1
DEBUG:cmdstanpy:runnin

KeyboardInterrupt: ignored

11:46:33 - cmdstanpy - ERROR - Chain [3] error: terminated by signal 2 Unknown error -2
ERROR:cmdstanpy:Chain [5] error: terminated by signal 2 Unknown error -2
ERROR:cmdstanpy:Chain [3] error: terminated by signal 2 Unknown error -2
11:46:33 - cmdstanpy - ERROR - Chain [6] error: terminated by signal 2 Unknown error -2
ERROR:cmdstanpy:Chain [6] error: terminated by signal 2 Unknown error -2
11:46:33 - cmdstanpy - ERROR - Chain [4] error: terminated by signal 2 Unknown error -2
ERROR:cmdstanpy:Chain [4] error: terminated by signal 2 Unknown error -2


In [ ]:
az.summary(ritorno['inference_data'], var_names=['theta','sigma','gamma_th', 'hyper_y_start_m', 'hyper_y_start_s', 'hyper_gamma_phi_m', 'hyper_gamma_phi_s','betas','w_s','rho','alpha'])

In [ ]:
az.plot_pair(ritorno['inference_data'], var_names=['hyper_gamma_phi_m', 'hyper_gamma_phi_s', 'theta', 'sigma'], divergences=True)

In [ ]:
az.plot_trace(ritorno['inference_data'], var_names=['phi','theta','sigma','betas','w_s','rho','alpha'], divergences=True)

In [ ]:
az.plot_posterior(ritorno['inference_data'], var_names=['theta','sigma','hyper_y_start_m','hyper_y_start_s', 'hyper_gamma_phi_m', 'hyper_gamma_phi_s','betas','w_s','rho','alpha'], hdi_prob=0.95)

In [ ]:
aux_shape = ritorno['inference_data'].posterior.phi.values.shape
sns.kdeplot(ritorno['inference_data'].posterior.phi.values.reshape((aux_shape[0]*aux_shape[1],aux_shape[2],aux_shape[3]))[:,0,:], legend=False)

In [ ]:
aux_shape = ritorno['inference_data'].posterior.phi.values.shape
sns.kdeplot(ritorno['inference_data'].posterior.phi.values.reshape((aux_shape[0]*aux_shape[1],aux_shape[2],aux_shape[3]))[:,1,:], legend=False)

In [ ]:
aux_shape = ritorno['inference_data'].posterior.c.values.shape
sns.kdeplot(ritorno['inference_data'].posterior.c.values.reshape((aux_shape[0]*aux_shape[1],aux_shape[2])), legend=False)

In [ ]:
y_post_pred = ritorno['inference_data'].posterior.y_post_pred.to_numpy()
y_post_pred = y_post_pred.reshape(y_post_pred.shape[0]*y_post_pred.shape[1], y_post_pred.shape[2], y_post_pred.shape[3])
y_post_pred.shape

In [ ]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from IPython.display import display

In [ ]:
@widgets.interact(stazione = df.columns)
def f(stazione):
    col_map = sns.light_palette((20, 75, 70), input='husl', as_cmap=True)

    plt.figure(figsize=(14,8))
    ax = plt.subplot(1,1,1)

    idx_stazione = df.columns.to_list().index(stazione)
    
    """
    sns.lineplot(np.transpose(y_post_pred[0:50,:,idx_stazione]))
    """

    lower_lim = np.zeros(len(df.index))
    upper_lim = np.zeros(len(df.index))    
    for i in range(len(df.index)):
        lower_lim[i] = np.percentile(y_post_pred[:,i,idx_stazione],2.5)
        upper_lim[i] = np.percentile(y_post_pred[:,i,idx_stazione],97.5)
    
    
#    sns.lineplot(lower_lim)
#    sns.lineplot(upper_lim)
    
    for i in range(len(df.index)):
        ax.add_patch(mplt.patches.Rectangle((i,lower_lim[i]),1,upper_lim[i]-lower_lim[i], fill=True, color=col_map(180)))

    sns.lineplot(df[stazione])    
    sns.lineplot(np.mean(y_post_pred[:,:,idx_stazione], axis=0))


    index = 0
    for line in ax.get_lines():
        if index == 0:
            col_map = sns.dark_palette((230,90,65), input='husl', as_cmap=True)
            line.set_c(col_map(155))
        else:
            col_map = sns.dark_palette((10,90,65), input='husl', as_cmap=True)
            line.set_c(col_map(175))
        index += 1
        
    plt.ylim(bottom=0,top=2.5)

    primo_giorno = datetime.date(2018,1,1)
    date_da_segnare = []
    date_da_segnare_posizioni = []

    for i in range(12):
        date_da_segnare.append(datetime.date(2018,i+1,1))
        date_da_segnare_posizioni.append((date_da_segnare[2*i] - primo_giorno).days)
        date_da_segnare.append(datetime.date(2018,i+1,15))
        date_da_segnare_posizioni.append((date_da_segnare[2*i+1] - primo_giorno).days)
        date_da_segnare[2*i] = date_da_segnare[2*i].isoformat()
        date_da_segnare[2*i+1] = date_da_segnare[2*i+1].isoformat()

    plt.xticks(date_da_segnare_posizioni,date_da_segnare,rotation=65)
    plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=True,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=True)
    plt.grid()
    
    """
    ax.get_legend().remove()
    col_vals = np.linspace(1,255,num=len(df.columns))
    index = 0
    for line in ax.get_lines():
        if(index == len(ax.get_lines()) - 1):
            col_map = sns.dark_palette((230,90,65), input='husl', as_cmap=True)
            line.set_c(col_map(175))
            continue
        if(index == len(ax.get_lines()) - 2):
            col_map = sns.dark_palette((120,90,65), input='husl', as_cmap=True)
            line.set_c(col_map(175))
            index += 1
            continue

        #line.set_c(col_map(int(np.round(col_vals[index]))))
        line.set_c(col_map(220))
        line.set_alpha(0.3)
        index += 1
    """
    
    

    plt.show()

In [ ]:
def f(stazione):
    col_map = sns.light_palette((20, 75, 70), input='husl', as_cmap=True)

    plt.figure(figsize=(14,8))
    ax = plt.subplot(1,1,1)

    idx_stazione = df.columns.to_list().index(stazione)
    
    """
    sns.lineplot(np.transpose(y_post_pred[0:50,:,idx_stazione]))
    """

    lower_lim = np.zeros(len(df.index))
    upper_lim = np.zeros(len(df.index))    
    for i in range(len(df.index)):
        lower_lim[i] = np.percentile(y_post_pred[:,i,idx_stazione],2.5)
        upper_lim[i] = np.percentile(y_post_pred[:,i,idx_stazione],97.5)
    
    
#    sns.lineplot(lower_lim)
#    sns.lineplot(upper_lim)
    
    for i in range(len(df.index)):
        ax.add_patch(mplt.patches.Rectangle((i,lower_lim[i]),1,upper_lim[i]-lower_lim[i], fill=True, color=col_map(180)))

    sns.lineplot(df[stazione])    
    sns.lineplot(np.mean(y_post_pred[:,:,idx_stazione], axis=0))


    index = 0
    for line in ax.get_lines():
        if index == 0:
            col_map = sns.dark_palette((230,90,65), input='husl', as_cmap=True)
            line.set_c(col_map(155))
        else:
            col_map = sns.dark_palette((10,90,65), input='husl', as_cmap=True)
            line.set_c(col_map(175))
        index += 1
        
    plt.ylim(bottom=0,top=2.5)

    primo_giorno = datetime.date(2018,1,1)
    date_da_segnare = []
    date_da_segnare_posizioni = []

    for i in range(12):
        date_da_segnare.append(datetime.date(2018,i+1,1))
        date_da_segnare_posizioni.append((date_da_segnare[2*i] - primo_giorno).days)
        date_da_segnare.append(datetime.date(2018,i+1,15))
        date_da_segnare_posizioni.append((date_da_segnare[2*i+1] - primo_giorno).days)
        date_da_segnare[2*i] = date_da_segnare[2*i].isoformat()
        date_da_segnare[2*i+1] = date_da_segnare[2*i+1].isoformat()

    plt.xticks(date_da_segnare_posizioni,date_da_segnare,rotation=65)
    plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=True,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=True)
    plt.grid()
    
    """
    ax.get_legend().remove()
    col_vals = np.linspace(1,255,num=len(df.columns))
    index = 0
    for line in ax.get_lines():
        if(index == len(ax.get_lines()) - 1):
            col_map = sns.dark_palette((230,90,65), input='husl', as_cmap=True)
            line.set_c(col_map(175))
            continue
        if(index == len(ax.get_lines()) - 2):
            col_map = sns.dark_palette((120,90,65), input='husl', as_cmap=True)
            line.set_c(col_map(175))
            index += 1
            continue

        #line.set_c(col_map(int(np.round(col_vals[index]))))
        line.set_c(col_map(220))
        line.set_alpha(0.3)
        index += 1
    """
    
    

    plt.show()

In [ ]:
f('CAORLE')

In [ ]:
f('SAVIGNANO DI RIGO')